In [1]:
# -*- coding: utf-8 -*-
"""
Model training pipeline for motion HAR (IMU)
"""

import os
import sys
from pathlib import Path
import random
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
from tqdm import tqdm
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import balanced_accuracy_score, f1_score
from sklearn.utils import class_weight
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (Input, Conv1D, BatchNormalization, MaxPooling1D, Dropout, Flatten, Dense, Activation, concatenate)
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import optimizers, backend as K

# Configuration
PROJECT_ROOT = Path(__file__).resolve().parent
sys.path.append(str(PROJECT_ROOT / ".." / ".." / ".." / "HCAR"))

# GPU configuration
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Seeds for reproducibility
SEED = 20
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Training settings
MODEL_VERSION = 36
BATCH_SIZE = 64
EPOCHS = 30
HAND = 'Right'
SUB_SR = 16000
IMU_SR = 50
WINDOW_LEN_IMU = 2 * IMU_SR      # 2 seconds
HOP_LEN_IMU    = int(0.2 * IMU_SR)  # 0.2s stride

# Model input shape
IMU_INPUT_SHAPE = (100, 9)

# Participants
TRAIN_PIDS = [10, 100, 101, 102, 103]
VALID_PIDS = [104]
TEST_PIDS = [3, 4]

# Paths
BASE_PATH = Path("../../")
DATA_PATH = BASE_PATH / "Data/Train_Data/3_MMExamples"
MODEL_SAVE_PATH = BASE_PATH / f"Models/tensorflow_model/Motion/Motion_ver{MODEL_VERSION}/{HAND}"
ACC_SAVE_PATH = BASE_PATH / f"../../Result/Model_Accuracy/Motion/Motion_ver{MODEL_VERSION}/{HAND}"
PRED_SAVE_PATH = BASE_PATH / f"../../Result/Model_Preds/Motion/Motion_ver{MODEL_VERSION}/{HAND}"
NORM_PATH = BASE_PATH / f"Normalization_params/Normalization_params_pickle/normalization_params_motion_{HAND}_ver{MODEL_VERSION}.pkl"

MODEL_SAVE_PATH.mkdir(parents=True, exist_ok=True)
ACC_SAVE_PATH.mkdir(parents=True, exist_ok=True)
PRED_SAVE_PATH.mkdir(parents=True, exist_ok=True)

# Functions
def frame(data: np.ndarray, window_length: int, hop_length: int) -> np.ndarray:
    # Frame the 1D/2D array into overlapping windows.
    if data.shape[0] < window_length:
        pad_len = window_length - data.shape[0]
        pad = np.zeros((pad_len, ) + data.shape[1:], dtype=data.dtype)
        data = np.concatenate([data, pad], axis=0)
    n_frames = 1 + (data.shape[0] - window_length) // hop_length
    shape = (n_frames, window_length) + data.shape[1:]
    strides = (data.strides[0] * hop_length,) + data.strides[1:]
    return np.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)

def compute_norm_params(train_ids: list) -> dict:
    all_segments = []
    for pid in train_ids:
        folder = DATA_PATH / str(pid) / HAND / str(SUB_SR)
        for file in tqdm(folder.glob('*.pkl'), desc=f"Norm params pid={pid}"):
            data = pickle.load(open(file, 'rb'))['IMU']
            if data.size:
                all_segments.append(data)
    concat = np.concatenate(all_segments, axis=0)
    flat = concat.reshape(-1, concat.shape[-1])
    return {
        'max': np.percentile(flat, 80, axis=0),
        'min': np.percentile(flat, 20, axis=0),
        'mean': flat.mean(axis=0),
        'std': flat.std(axis=0)
    }

def normalize_imu(data: np.ndarray, params: dict) -> np.ndarray:
    # Normalize IMU data to [-1,1] then standardize to zero mean and unit std.

    pmax = params['max'].reshape(1,1,-1)
    pmin = params['min'].reshape(1,1,-1)
    mean = params['mean'].reshape(1,1,-1)
    std  = params['std'].reshape(1,1,-1)
    scaled = 1 + (data - pmax) * 2 / (pmax - pmin)
    return (scaled - mean) / std

def load_dataset(pids: list, params: dict) -> tuple:
    # Load and normalize IMU & audio data, return arrays and labels.
    X_imu, y = [], []
    for pid in pids:
        folder = DATA_PATH / str(pid) / HAND / str(SUB_SR)
        for file in tqdm(folder.glob('*.pkl'), desc=f"Load pid={pid}"):
            meta = file.stem.split('---')
            _, activity, _ = meta
            data = pickle.load(open(file,'rb'))
            imu = data['IMU'].astype(np.float32)

            if imu.size:
                imu = normalize_imu(imu, params)
                X_imu.append(np.concatenate(imu, axis=0))
                y.extend([[pid, activity]] * imu.shape[0])
    return (
        np.concatenate(X_imu, axis=0),
        np.array(y)
    )

def data_generator(X_imu, y, batch_size, shuffle=True):
    num_samples = X_imu.shape[0]
    while True:
        if shuffle:
            indices = np.arange(num_samples)
            np.random.shuffle(indices)
        else:
            indices = np.arange(num_samples)
        for start in range(0, num_samples, batch_size):
            end = start + batch_size
            batch_ids = indices[start:end]
            yield X_imu[batch_ids], y[batch_ids]

# Create IMU model
def create_motion_model():

    inputs = Input(shape=IMU_INPUT_SHAPE, name="IMU_input")
    x = Conv1D(filters=128, kernel_size=10, strides=1, padding='valid', activation='relu', name='Conv_1')(inputs)
    x = BatchNormalization(name='batch_normalization_1')(x)

    x = Conv1D(filters=128, kernel_size=10, strides=1, activation='relu', name='Conv_2')(x)
    x = BatchNormalization(name='batch_normalization_2')(x)
    x = MaxPooling1D(pool_size=2, name='Max_pool_1')(x)

    x = Conv1D(filters=256, kernel_size=10, strides=1, activation='relu', name='Conv_3')(x)
    x = BatchNormalization(name='batch_normalization_3')(x)

    x = Conv1D(filters=256, kernel_size=10, strides=1, activation='relu', name='Conv_4')(x)
    x = BatchNormalization(name='batch_normalization_4')(x)
    x = MaxPooling1D(pool_size=2, name='Max_pool_2')(x)
    x = Dropout(rate=0.5, name='Dropout_1')(x)

    x = Flatten(name='flatten')(x)

    x = Dense(8192, activation='relu', name='dense_1')(x)
    x = Dense(1024, activation='relu', name='dense_2')(x)
    x = Dense(256, activation='relu', name='dense_3')(x)

    outputs = Dense(6, activation='softmax', name='final_output')(x)

    motion_model = Model(inputs=inputs, outputs=outputs, name='IMU_model')
    adamOpt = optimizers.Adam(learning_rate=0.001)
    motion_model.compile(optimizer=adamOpt, loss='categorical_crossentropy', metrics=['accuracy'])

    return motion_model

def evaluate_and_save(model: Model, X_imu, y_labels, lb: LabelBinarizer):
    # Perform filewise prediction, compute metrics, save results.
    preds = model.predict(X_imu, batch_size=BATCH_SIZE)
    df = pd.DataFrame(preds, columns=lb.classes_)
    df['file_name'] = y_labels[:,1]
    df['y_pred'] = df.drop(columns=['file_name']).idxmax(axis=1)
    df['y_true'] = df['file_name']

    ba = balanced_accuracy_score(df['y_true'], df['y_pred'])
    f1 = f1_score(df['y_true'], df['y_pred'], average='weighted')
    return df, ba, f1

# Main Execution
def main():
    # Compute or load normalization params
    if NORM_PATH.exists():
        params = pickle.load(open(NORM_PATH,'rb'))
    else:
        params = compute_norm_params(TRAIN_PIDS)
        pickle.dump(params, open(NORM_PATH,'wb'))

    # Load datasets
    X_imu_tr, y_tr = load_dataset(TRAIN_PIDS, params)
    X_imu_va, y_va = load_dataset(VALID_PIDS, params)
    X_imu_te, y_te = load_dataset(TEST_PIDS, params)

    # Encode labels and compute class weights
    lb = LabelBinarizer().fit(y_tr[:,1])
    y_tr_lbl = lb.transform(y_tr[:,1])
    y_va_lbl = lb.transform(y_va[:,1])
    cw = class_weight.compute_class_weight('balanced', classes=lb.classes_, y=y_tr[:,1])
    cw_dict = dict(enumerate(cw))

    # Build model
    motion_model = create_motion_model()

    # Train
    reduce_lr = ReduceLROnPlateau('val_loss',0.1,3,verbose=1,min_lr=1e-6)
    early_stop = EarlyStopping('val_loss',5,verbose=1,restore_best_weights=True)
    train_gen = data_generator(X_imu_tr, y_tr_lbl, BATCH_SIZE, shuffle=True)
    val_gen = data_generator(X_imu_va, y_va_lbl, BATCH_SIZE, shuffle=False)

    motion_model.fit(
        train_gen,
        epochs=EPOCHS,
        validation_data=val_gen,
        callbacks=[reduce_lr, early_stop],
        class_weight=cw_dict,
        verbose=1
    )

    # Save model and accuracy
    motion_model.save(MODEL_SAVE_PATH / 'Motion_Scratch.h5')

    # Evaluate
    results_df, ba, f1 = evaluate_and_save(motion_model, X_imu_te, y_te, lb)
    results_df.to_csv(PRED_SAVE_PATH / f'{HAND}.csv', index=False)

    with open(ACC_SAVE_PATH / 'MM_Scratch.txt', 'w') as f:
        f.write(f'Balanced Accuracy: {ba}\nF1 Score: {f1}\n')

    # Cleanup
    del motion_model, X_imu_tr, y_tr, X_imu_va, y_va, X_imu_te, y_te
    gc.collect()

if __name__ == '__main__':
    main()
